In [1]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import numpy as np
import math
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential
from keras.layers import Dropout, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D, Dense, Activation
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [2]:
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
ROWS = 360  #720
COLS = 640 #1280
CHANNELS = 3

In [3]:
def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (COLS, ROWS), interpolation=cv2.INTER_CUBIC)
    return im

In [4]:
optimizer = RMSprop(lr=1e-4)
objective = 'categorical_crossentropy'

def center_normalize(x):
    return (x - K.mean(x)) / K.std(x)

model = Sequential()

model.add(Activation(activation=center_normalize, input_shape=(ROWS, COLS, CHANNELS)))

model.add(Convolution2D(32, 5, 5, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(32, 5, 5, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))

model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))

model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))

model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))


model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(FISH_CLASSES)))
model.add(Activation('sigmoid'))

E:\Anaconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), data_format="channels_last", padding="same", activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
E:\Anaconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), data_format="channels_last", padding="same", activation="relu")`
  if sys.path[0] == '':
E:\Anaconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_last")`
  del sys.path[0]
E:\Anaconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), data_format="channels_last", padding="same", activation="relu")`
  from ipykernel import kernelapp as app
E:\Anaco

In [5]:
model.load_weights('weights.hdf5')

In [6]:
TEST_DIR_STG1 = 'F:/fisheries/test_stg1/test_stg1/'
test_files_stg1 = [im for im in os.listdir(TEST_DIR_STG1)]
test_files = [im for im in os.listdir(TEST_DIR_STG1)]

TEST_DIR_STG2 = 'F:/fisheries/test_stg2/'
test_files_stg2 = [im for im in os.listdir(TEST_DIR_STG2)]
test_files = test_files + [ 'test_stg2/'+im for im in os.listdir(TEST_DIR_STG2)]

test = np.ndarray((len(test_files), ROWS, COLS, CHANNELS), dtype=np.uint8)

In [7]:
i=0
for im in test_files_stg1:
    test[i] = read_image(TEST_DIR_STG1+im)
    i=i+1

for im in test_files_stg2:
    test[i] = read_image(TEST_DIR_STG2+im)
    i=i+1

In [8]:
test_preds= model.predict(test, batch_size = 8, verbose=1)


13153/13153 [==============================] - 408s 31ms/step


In [9]:
submission_all = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission_all.insert(0, 'image', test_files)
submission_all.to_csv("submission_all_4.csv")